In [ ]:

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


ridership = pd.read_csv("./../Data/CTA-Ridership-Daily_Boarding_Totals_20240829.csv", parse_dates=["service_date"])


display(ridership)

# Setting date column as index
ridership = ridership.sort_values("service_date").set_index("service_date")

# Checks for the modification into the dataset
display(ridership)

# Drops the calculated column "total_rides" as this is just element-wise addition from columns "bus" and "rail_boardings".
ridership = ridership.drop("total_rides", axis=1)



# remove duplicate observations, if any
ridership = ridership.drop_duplicates()


ridership.shape



# Looks at the first few months of 2019
ridership["2019-03":"2019-05"].plot(grid=True, marker=".", figsize=(8, 3.5))
plt.show()


### Naive Forecasting
# Creates a 7-day differencing time-series out of difference between two time-series - the original one
# and other one lagged by one week
diff_7 = ridership[["bus", "rail_boardings"]].diff(7)["2019-03":"2019-05"]


# First shows the two overlayed time-series

# Prepares a figure with two plots - one for the overlayed time-series and 
# other is for the differencing time-series
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 5))

# Plots the original time-series in the first axis
ridership.plot(ax=axs[0], legend=False, marker=".")  

# Plots the lagged time-series in the first axis as overlayed
ridership.shift(7).plot(ax=axs[0], grid=True, legend=False, linestyle=":")

# Plots the differencing time-series prepared earlier in the next axis
diff_7.plot(ax=axs[1], grid=True, marker=".")

# [OPTIONAL] Sets y-axis limit of the first plot to zoom into range of interest
axs[0].set_ylim([170_000, 900_000])

# Shows the prepared plot
plt.show()


# Measures the performance of naive forecasting over metric Mean absolute error (MAE), also called mean absolute deviation (MAD)
diff_7.abs().mean()


# The same performance is also expressed in Mean Absolute Percentage Error (MAPE)
(diff_7 / ridership[["bus", "rail_boardings"]]["2019-03":"2019-05"]).abs().mean()

The naive forecasts give a MAPE of roughly **8.3%** for bus and **9.0%** for rail boardings.


### Univariate Forecasting
# Splits the time-series into three periods, for training, validation and testing
# The values are scaled down by a factor of one million, to ensure the values are near the 0–1 range
rail_train = ridership["rail_boardings"]["2016-01":"2018-12"] / 1e6  # 3 years
rail_val = ridership["rail_boardings"]["2019-01":"2019-05"] / 1e6    # 5 months
rail_test = ridership["rail_boardings"]["2019-06":] / 1e6            # remaining period from 2019-06


# Prepares TensorFlow specific datasets

seq_length = 56    # represents sequence of past 8 weeks (56 days) of ridership data

tf.random.set_seed(42)

rail_train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,  # shuffling in training windows is recommended for gradient descent optimizer
    seed=42
)

rail_val_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_val.to_numpy(),
    targets=rail_val[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=False  # shuffling is not required for any testing data including validation data
)


#### Forecasting using A Linear Model


tf.random.set_seed(42)

# Creates a linear model over a dense layer
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=[seq_length])])

# Sets callback to stop training when model does improve after a certain number of training iterations
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae", patience=50, restore_best_weights=True)

# Sets the model optimizer and compiles it with specific loss function and metric
model.compile(
    loss=tf.keras.losses.Huber(),   # Huber loss works well in minimizing MAE
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9), 
    metrics=["mae"])

# Starts model training process over specified training, validation data and callbacks
history = model.fit(rail_train_ds, 
                    validation_data=rail_val_ds, 
                    epochs=500,
                    callbacks=[early_stopping_callback])


#### Forecasting Using a Simple RNN

# Resets all the keras states
tf.keras.backend.clear_session()

tf.random.set_seed(42)

# Creates an RNN with 32 recurrent neurons followed by a dense output layer with one output neuron
univar_simple_rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
    tf.keras.layers.Dense(1)
])



# Sets callback to stop training when model does improve after a certain number of training iterations
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae", patience=50, restore_best_weights=True)

# Sets the model optimizer and compiles it with specific loss function and metric
univar_simple_rnn.compile(
    loss="huber",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9),
    metrics=["mae"])

# Starts model training process over specified training, validation data and callbacks
history = univar_simple_rnn.fit(
    rail_train_ds, validation_data=rail_val_ds, epochs=500, callbacks=[early_stopping_callback])




# After training, model gets evaluated against validation data

val_loss, val_mae = univar_simple_rnn.evaluate(rail_val_ds)
print("Validation MAE of the Simple RNN:", val_mae * 1e6)


The model performance shown above indicates that the Simple RNN worked [MAE: 29935] better than the baseline model [MAE: 42143] and the linear model [MAE: 37060].



#### Forecasting Using a Deep RNN

# Resets all the keras states
tf.keras.backend.clear_session()

tf.random.set_seed(42)

# Creates a Deep RNN with multiple layers of simple RNN each with 32 recurrent neurons 
# followed by a dense output layer with one output neuron
univar_deep_rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),  # sequence-to-sequence layer
    tf.keras.layers.SimpleRNN(32, return_sequences=True),                         # sequence-to-sequence layer
    tf.keras.layers.SimpleRNN(32),                                                # sequence-to-vector layer
    tf.keras.layers.Dense(1)
])


# Sets callback to stop training when model does improve after a certain number of training iterations
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae", patience=50, restore_best_weights=True)

# Sets the model optimizer and compiles it specific loss function and metric
univar_deep_rnn.compile(
    loss="huber",
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    metrics=["mae"])

# Starts model training process over specified training, validation data and callbacks
history = univar_deep_rnn.fit(
    rail_train_ds, validation_data=rail_val_ds, epochs=500, callbacks=[early_stopping_callback])




# After training, model gets evaluated against validation data

val_loss, val_mae = univar_deep_rnn.evaluate(rail_val_ds)
print("Validation MAE of the Deep RNN:", val_mae * 1e6)




The above performance shows that deep RNN is better than baseline model [MAE: 42143] and the linear model [MAE: 37060], but it couldn't beat simple RNN [MAE: 29935]. Hence, deep RNN might not be a good fit for this case. Now, let's experiment with multivariate forecasting to check if it can improve model performance.